# AutoEncoder-異常檢測
**注意，本篇模型與相關設定用於demo，整體準確度較差，若有需求，可更改模型與調整訓練參數，並使用GPU進行訓練**

## 宣告套件庫（若使用GPU,請將"#device = ('cuda')"，中的"cpu"改為"cuda"）

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Subset
from google.colab.patches import cv2_imshow
import numpy as np
device = ('cpu')

## 宣告模型（可適度加深，或是使用CNN）

In [3]:

# 定義自編碼器模型
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        
        # 編碼器（壓縮）
        self.encoder = nn.Sequential(
            nn.Linear(784, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
        )
        
        # 解碼器（重構）
        self.decoder = nn.Sequential(
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 784),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded




## 宣告訓練參數，將0設定為正常，其餘為異常

In [6]:
# 設置訓練參數
batch_size = 128
learning_rate = 0.001
num_epochs = 10

# 載入手寫數字資料集
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)

# 只選擇正常類別（0）進行訓練
train_indices = [i for i, (_, label) in enumerate(train_dataset) if ((label == 0))]
train_subset = Subset(train_dataset, train_indices)

train_loader = torch.utils.data.DataLoader(train_subset, batch_size=batch_size, shuffle=True)

# 初始化模型和損失函數
model = Autoencoder().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


## 開始訓練

In [7]:
# 訓練模型
for epoch in range(num_epochs):
    running_loss = 0.0
    
    for data in train_loader:
        images, _ = data
        
        # 將圖像展平為一維張量
        images = images.view(images.size(0), -1).to(device)
        
        # 正向傳遞
        outputs = model(images)
        loss = criterion(outputs, images)
        
        # 反向傳遞和優化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    # 計算每個epoch的平均損失
    average_loss = running_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Average Loss: {average_loss:.4f}')

print('訓練完成！')

Epoch [1/10], Average Loss: 0.9273
Epoch [2/10], Average Loss: 0.7840
Epoch [3/10], Average Loss: 0.7622
Epoch [4/10], Average Loss: 0.7399
Epoch [5/10], Average Loss: 0.7186
Epoch [6/10], Average Loss: 0.6959
Epoch [7/10], Average Loss: 0.6779
Epoch [8/10], Average Loss: 0.6709
Epoch [9/10], Average Loss: 0.6676
Epoch [10/10], Average Loss: 0.6642
訓練完成！


## 測試模型

In [8]:

model.eval()
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=True)
model.to('cpu')
with torch.no_grad():
    normal_samples = []
    anomaly_samples = []
    
    for i,(images, labels) in enumerate(test_loader):
        # 將圖像展平為一維張量
        images = images.view(images.size(0), -1)
        
        # 正向傳遞
        outputs = model(images)
        outputs=outputs.view(outputs.size(0), 1, 28, 28)
        images=images.view(images.size(0), 1, 28, 28)
        print('------')
        print('origin')
        cv2_imshow(np.array(images[0][0])*255)
        print('output')
        cv2_imshow(np.array(outputs[0][0])*255)

        if i>30:
            break

------
origin


output


------
origin


output


------
origin


...

output


------
origin


output


------
origin


output
